<a href="https://colab.research.google.com/github/nemodesouza/TCC-AIX/blob/main/PA2_consistencia_interna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# UNIVERSIDADE FEDERAL DE SANTA CATARINA
# CENTRO TECNOLÓGICO
# DEPARTAMENTO DE INFORMÁTICA E ESTATÍSTICA
# GRADUAÇÃO EM SISTEMAS DE INFORMAÇÃO
# TRABALHO DE CONCLUSÃO DE CURSO
# NEMO DE SOUZA

# PA2 = Análise de Consistência Interna do Checklist AIX
# Script para a análise de consistência interna do checklist de heurísticas de usabilidade

# Carregamento de bibliotecas
if (!require(psych)) install.packages("psych")      # Para alfa de Cronbach e análise fatorial
if (!require(dplyr)) install.packages("dplyr")      # Para manipulação de dados
if (!require(tidyr)) install.packages("tidyr")      # Para reorganização de dados
if (!require(ggplot2)) install.packages("ggplot2")  # Para visualizações

library(psych)
library(dplyr)
library(tidyr)
library(ggplot2)

In [ ]:
# Carregamento dos dados

### OBS: antes de executar, carregue os arquivos (de mesmo nome) na pasta do Colab (barra lateral "Arquivos")

# Ajuste o caminho conforme necessário
dados_path <- "PA2 a PA4 - matrix_avaliacao_aplicativos.csv"
heuristicas_path <- "matriz_heuristicas_e_itens_checklist.csv"

dados <- read.csv(dados_path, stringsAsFactors = FALSE)
heuristicas <- read.csv(heuristicas_path, stringsAsFactors = FALSE)

# Verificação da estrutura dos dados
str(dados)
head(dados)

# Verificar a estrutura das heurísticas
str(heuristicas)
head(heuristicas)

In [ ]:
#### Preparação dos dados para análise

# Seleção das colunas dos itens (c1 a c23)
itens <- dados[, grep("^c\\d+$", names(dados))]

# Conversão das respostas para valores numéricos (Sim = 1, Não = 0, NA = NA)
itens_num <- as.data.frame(lapply(itens, function(x) {
  ifelse(x == "Sim", 1, ifelse(x == "Não", 0, NA))
}))

# Verificação da conversão
head(itens_num)


In [ ]:
#### Criação do dicionário para mapear códigos de itens para descrições de heurísticas

# Estrutura arquivo:
# - ID Heurística: h1, h2, etc.
# - Heurística: Descrição da heurística
# - ID checklist: c1, c2, etc.
# - Item de checklist: Descrição do checklist

id_heuristicas <- heuristicas$ID.Heurística
descricoes_heuristicas <- heuristicas$Heurística
codigos_itens <- heuristicas$ID.checklist
descricoes_itens <- heuristicas$Item.de.checklist

# Criação dataframe de mapeamento
dicionario <- data.frame(
  codigo_item = codigos_itens,
  descricao_item = descricoes_itens,
  id_heuristica = id_heuristicas,
  descricao_heuristica = descricoes_heuristicas,
  stringsAsFactors = FALSE
)

# Verificação o dicionário criado
head(dicionario)

In [ ]:
#### Identificação e remoção de colunas problemáticas
# Cálculo da porcentagem de NAs em cada coluna
na_percentagem <- colMeans(is.na(itens_num)) * 100
print("Porcentagem de NAs em cada coluna:")
print(na_percentagem)

# Cálculo do desvio padrão de cada coluna
desvios <- sapply(itens_num, sd, na.rm = TRUE)
print("Desvio padrão de cada coluna:")
print(desvios)

# Colunas com desvio padrão zero ou muito baixo
colunas_desvio_zero <- names(desvios[desvios < 0.1])
print("Colunas com desvio padrão próximo de zero:")
print(colunas_desvio_zero)

# Colunas com muitos NAs (mais de 30%)
colunas_muitos_na <- names(na_percentagem[na_percentagem > 30])
print("Colunas com mais de 30% de NAs:")
print(colunas_muitos_na)

# Colunas a serem removidas (desvio zero ou muitos NAs)
colunas_remover <- unique(c(colunas_desvio_zero, colunas_muitos_na))
print("Colunas a serem removidas:")
print(colunas_remover)

# Conjunto de dados limpo
itens_limpos <- itens_num[, !(names(itens_num) %in% colunas_remover)]

# Verificação das dimensões do conjunto de dados após limpeza
print("Dimensões do conjunto de dados após limpeza:")
print(dim(itens_limpos))

# Dataframe com informações detalhadas sobre os itens removidos
itens_removidos <- data.frame(
  codigo = colunas_remover,
  descricao = sapply(colunas_remover, function(codigo) {
    idx <- which(dicionario$codigo_item == codigo)
    if (length(idx) > 0) dicionario$descricao_item[idx[1]] else NA
  }),
  motivo = sapply(colunas_remover, function(codigo) {
    if (codigo %in% colunas_desvio_zero) {
      if (codigo %in% colunas_muitos_na) {
        return("Desvio padrão próximo de zero e muitos NAs")
      } else {
        return("Desvio padrão próximo de zero")
      }
    } else {
      return("Muitos valores NA")
    }
  })
)


In [ ]:
# Informações detalhadas sobre os itens removidos
print("Detalhes dos itens removidos:")
print(itens_removidos)

In [ ]:
# Salvamento da lista de itens removidos para referência futura
write.csv(itens_removidos, "itens_removidos.csv", row.names = FALSE)
# OBS: os arquivos gerados estarão disponíveis para download na lateral, no menu "Arquivos"

In [ ]:
#### Calculo do Alfa de Cronbach usando correlações par a par
alfa_resultado <- alpha(itens_limpos, check.keys = TRUE, use = "pairwise.complete.obs")
print("Alfa de Cronbach para o checklist limpo (usando correlações par a par):")
print(alfa_resultado$total$raw_alpha)
print(alfa_resultado$total)

In [ ]:
#### Calculo do Ômega de McDonald  com correlações par a par
tryCatch({
  # Tentar com opções mais robustas
  omega_resultado <- omega(itens_limpos, nfactors = 4,
                           use = "pairwise.complete.obs",
                           fm = "ml",       # Método de máxima verossimilhança
                           plot = FALSE,    # Não plotar resultados
                           warnings = FALSE) # Suprimir avisos

  cat("Ômega de McDonald para o checklist limpo:\n")
  cat("Ômega Total:", round(omega_resultado$omega.tot, 3), "\n")
  cat("Ômega Hierárquico:", round(omega_resultado$omega_h, 3), "\n")
}, error = function(e) {
  cat("Erro ao calcular Ômega de McDonald com 4 fatores:", e$message, "\n")
  cat("Tentando com menos fatores...\n")

  # Tentar com 3 fatores
  tryCatch({
    omega_resultado <- omega(itens_limpos, nfactors = 3,
                             use = "pairwise.complete.obs",
                             fm = "ml",
                             plot = FALSE,
                             warnings = FALSE)

    cat("Ômega Total (3 fatores):", round(omega_resultado$omega.tot, 3), "\n")
    cat("Ômega Hierárquico (3 fatores):", round(omega_resultado$omega_h, 3), "\n")

    # Armazenar o resultado para uso posterior
    assign("omega_resultado", omega_resultado, envir = .GlobalEnv)
  }, error = function(e2) {
    cat("Erro ao calcular Ômega de McDonald com 3 fatores:", e2$message, "\n")
    cat("Tentando com 2 fatores...\n")

    # Tentar com 2 fatores
    tryCatch({
      omega_resultado <- omega(itens_limpos, nfactors = 2,
                               use = "pairwise.complete.obs",
                               fm = "ml",
                               plot = FALSE,
                               warnings = FALSE)

      cat("Ômega Total (2 fatores):", round(omega_resultado$omega.tot, 3), "\n")
      cat("Ômega Hierárquico (2 fatores):", round(omega_resultado$omega_h, 3), "\n")

      # Armazenar o resultado para uso posterior
      assign("omega_resultado", omega_resultado, envir = .GlobalEnv)
    }, error = function(e3) {
      cat("Erro ao calcular Ômega de McDonald com 2 fatores:", e3$message, "\n")

      # Última tentativa: usar um método alternativo
      cat("Tentando método alternativo (minres)...\n")
      tryCatch({
        omega_resultado <- omega(itens_limpos, nfactors = 2,
                                 use = "pairwise.complete.obs",
                                 fm = "minres",  # Método alternativo
                                 plot = FALSE,
                                 warnings = FALSE)

        cat("Ômega Total (método alternativo):", round(omega_resultado$omega.tot, 3), "\n")
        cat("Ômega Hierárquico (método alternativo):", round(omega_resultado$omega_h, 3), "\n")

        # Armazenar o resultado para uso posterior
        assign("omega_resultado", omega_resultado, envir = .GlobalEnv)
      }, error = function(e4) {
        cat("Não foi possível calcular o Ômega de McDonald com nenhum método.\n")
        cat("Prosseguindo com a análise sem o Ômega.\n")
      })
    })
  })
})


In [ ]:
#### Cálculo do Alfa de Cronbach para cada grupo de heurísticas

# Grupos únicos de heurísticas
grupos_unicos <- unique(dicionario$id_heuristica[!is.na(dicionario$id_heuristica)])
resultados_por_grupo <- list()

print("Alfa de Cronbach por grupo de heurísticas:")

for (grupo in grupos_unicos) {
  # Obtenção dos códigos dos itens para este grupo
  codigos_grupo <- dicionario$codigo_item[dicionario$id_heuristica == grupo]

  # Verificação de se todos os códigos existem nas colunas de itens limpos
  codigos_validos <- codigos_grupo[codigos_grupo %in% names(itens_limpos)]

  # Se pelo menos 2 itens válidos, calculo do alfa
  if (length(codigos_validos) > 1) {

    # Selação apenas desses itens dos dados
    itens_grupo <- itens_limpos[, codigos_validos]

    # Cálculo do Alfa de Cronbach para esse grupo
    alfa_grupo <- alpha(itens_grupo, check.keys = TRUE, use = "pairwise.complete.obs")

    # Nome do grupo (pegar a primeira ocorrência não-NA)
    nome_grupo <- dicionario$descricao_heuristica[dicionario$id_heuristica == grupo][1]

    # Armazenar resultados
    resultados_por_grupo[[grupo]] <- list(
      nome = nome_grupo,
      alfa_raw = alfa_grupo$total$raw_alpha,
      alfa_std = alfa_grupo$total$std.alpha,
      n_itens = length(codigos_validos),
      itens = codigos_validos
    )

    # Exibição de resultados
    cat("\nGrupo:", grupo, "-", nome_grupo, "\n")
    cat("Número de itens:", length(codigos_validos), "\n")
    cat("Itens:", paste(codigos_validos, collapse = ", "), "\n")
    cat("Alfa de Cronbach (raw):", round(alfa_grupo$total$raw_alpha, 3), "\n")
    cat("Alfa de Cronbach (std):", round(alfa_grupo$total$std.alpha, 3), "\n")
  } else {
    cat("\nGrupo:", grupo, "\n")
    cat("Número de itens insuficiente para calcular alfa (precisa de pelo menos 2 itens)\n")
    cat("Itens:", paste(codigos_validos, collapse = ", "), "\n")
  }
}


In [ ]:
#### Análise de itens - como cada item afeta a confiabilidade
print("Alfa se o item for removido:")
alfa_drop <- alfa_resultado$alpha.drop

# Adicionar descrições dos itens
alfa_drop$descricao <- sapply(rownames(alfa_drop), function(codigo) {
  idx <- which(dicionario$codigo_item == codigo)
  if (length(idx) > 0) dicionario$descricao_item[idx[1]] else NA
})

alfa_drop$heuristica <- sapply(rownames(alfa_drop), function(codigo) {
  idx <- which(dicionario$codigo_item == codigo)
  if (length(idx) > 0) dicionario$id_heuristica[idx[1]] else NA
})

# Ordenação do impacto na confiabilidade (maior alfa se removido = item menos importante)
alfa_drop_ordenado <- alfa_drop[order(alfa_drop$raw_alpha, decreasing = TRUE), ]
print(alfa_drop_ordenado)

# Identificação de itens problemáticos (remoção aumentaria o alfa)
itens_problematicos <- alfa_drop_ordenado[alfa_drop_ordenado$raw_alpha > alfa_resultado$total$raw_alpha, ]
if (nrow(itens_problematicos) > 0) {
  print("Itens cuja remoção aumentaria o Alfa de Cronbach:")
  print(itens_problematicos)
} else {
  print("Nenhum item cuja remoção aumentaria o Alfa de Cronbach.")
}

In [ ]:
#### Salvamento dos resultados em CSV

# Dataframe com os resultados por grupo
grupos_df <- data.frame(
  grupo = names(resultados_por_grupo),
  nome_grupo = sapply(resultados_por_grupo, function(x) x$nome),
  alfa = sapply(resultados_por_grupo, function(x) x$alfa_raw),  # IMPORTANTE: Usando 'alfa' consistentemente
  alfa_std = sapply(resultados_por_grupo, function(x) x$alfa_std),
  n_itens = sapply(resultados_por_grupo, function(x) x$n_itens)
)

# Filtragem dos grupos com pelo menos 2 itens
grupos_df <- grupos_df[grupos_df$n_itens > 1, ]

# Ordenação por valor de alfa
grupos_df <- grupos_df[order(grupos_df$alfa, decreasing = TRUE), ]  # IMPORTANTE: Usando 'alfa' consistentemente

# Criação de dataframe de resultados finais
resultados_csv <- data.frame(
  grupo = c("Checklist Completo", grupos_df$grupo),
  nome_grupo = c("Checklist Completo", grupos_df$nome_grupo),
  alfa = c(alfa_resultado$total$raw_alpha, grupos_df$alfa),  # IMPORTANTE: Usando 'alfa' consistentemente
  alfa_std = c(alfa_resultado$total$std.alpha, grupos_df$alfa_std),
  n_itens = c(ncol(itens_limpos), grupos_df$n_itens)
)

# Adição do ômega se calculado com sucesso
if (exists("omega_resultado")) {
  resultados_csv$omega_total <- c(omega_resultado$omega.tot, rep(NA, nrow(grupos_df)))
  resultados_csv$omega_hierarquico <- c(omega_resultado$omega_h, rep(NA, nrow(grupos_df)))
}


In [ ]:
# Salvamento em CSV
write.csv(resultados_csv, "resultados_consistencia_interna.csv", row.names = FALSE)
write.csv(alfa_drop_ordenado, "analise_itens.csv", row.names = TRUE)
write.csv(itens_removidos, "itens_removidos.csv", row.names = FALSE)

In [ ]:
# Matriz de correlação
tryCatch({
  cor_itens <- cor(itens_limpos, use = "pairwise.complete.obs")
  write.csv(cor_itens, "matriz_correlacao.csv", row.names = TRUE)
}, error = function(e) {
  print("Não foi possível calcular a matriz de correlação:", e$message)
})

In [ ]:
#### Visualizações

# 1. Visualização da correlação entre itens
cor_itens <- cor(itens_limpos, use = "pairwise.complete.obs")
heatmap(cor_itens,
        main = "Correlação entre itens do checklist",
        col = colorRampPalette(c("blue", "white", "red"))(100))


In [ ]:
# 2. Gráfico de barras para o Alfa por grupo
# IMPORTANTE: Usando 'alfa' consistentemente em todo o código
plot_alfa_grupo <- ggplot(grupos_df, aes(x = reorder(grupo, alfa), y = alfa)) +
  geom_bar(stat = "identity", fill = "steelblue") +
  geom_text(aes(label = round(alfa, 2)), hjust = -0.1, size = 3.5) +
  geom_text(aes(label = paste0("(", n_itens, " itens)")), y = 0.05, hjust = 0, size = 3) +
  coord_flip() +
  labs(title = "Alfa de Cronbach por Grupo de Heurísticas",
       x = "Grupo de Heurísticas",
       y = "Alfa de Cronbach") +
  theme_minimal() +
  geom_hline(yintercept = 0.7, linetype = "dashed", color = "red") +
  annotate("text", x = 1, y = 0.72, label = "Limiar de aceitabilidade (0.7)", size = 3.5)

# Plotar a imagem
print(plot_alfa_grupo)

In [ ]:
# 3. Gráfico para itens problemáticos
if(nrow(itens_problematicos) > 0) {

  itens_problematicos$item_nome <- paste0(rownames(itens_problematicos), ": ",
                                         substr(itens_problematicos$descricao, 1, 30), "...")

  plot_itens_problematicos <- ggplot(itens_problematicos, aes(x = reorder(item_nome, raw_alpha), y = raw_alpha)) +
    geom_bar(stat = "identity", fill = "coral") +
    geom_hline(yintercept = alfa_resultado$total$raw_alpha, linetype = "dashed", color = "blue") +
    annotate("text", x = 1, y = alfa_resultado$total$raw_alpha + 0.01,
             label = paste("Alfa atual:", round(alfa_resultado$total$raw_alpha, 3)), size = 3.5) +
    coord_flip() +
    labs(title = "Itens cuja remoção aumentaria o Alfa de Cronbach",
         x = "Item",
         y = "Alfa se o item for removido") +
    theme_minimal()


  print(plot_itens_problematicos)
}